# The Art of Prolog
1986年MIT Pressから出版されたPrologの教科書 The Art of Prolog は、私の前職である株式会社 構造計画研究所から1988年1月1日に「Prologの技芸」というタイトルで日本語訳が出版されました。

残念ながらPrologの技芸は絶版となり、Amazonの古本でも12,800円と高額で販売されています。

幸運なことに、英語の原本の方は、MIT Book pressのサーバからダウンロードすることができます。
- https://mitp-content-server.mit.edu/books/content/sectbyfn/books_pres_0/1407/1407.pdf?dl=1

2024年になって、推論エンジンの原理と応用を再度確認するために、「Prologの技芸」の例題をpythonにポーティングした推論エンジンと簡易パーサを使ってトレースしました。

さらに、swi prologを使って、23章の「コンパイラ」をトレースしました。以下のノートはその時の記録に加筆し、整理したものです。

## ノートブック実行の注意
最初に、コマンドパレットを表示し、"Select Notebook Kernel"を実行し、"Prolog"を選択します。
これで、code cellの内容がswi prologによって処理されます。

## 差分リストの表現
23章のコンパイラーでは、16章に出てくる差分リスト表現（X\Y）が使われていいますが、これを[X, Y]としても実行可能かどうかを試します。

差分リストのメリットは、リストのコピーが不要で、tail(尾部)は、ポインターとして働くことです。

## １６章　限定節文法による構文解析

### 文脈自由文法
16.1の文脈自由文法を定義します。

気を付ける点は、終端記号はカギ括弧で括る。
トップダウン文法でさりげなく、右再帰構文になっていることに注意！

In [1]:
% Figure 16.1 簡単な文脈自由文法
sentence --> noun_phrase, verb_phrase.
noun_phrase --> determiner, noun_phrase2.
noun_phrase --> noun_phrase2.
noun_phrase2 --> adjective, noun_phrase2.
noun_phrase2 --> noun.
verb_phrase --> verb.
verb_phrase --> verb, noun_phrase.
% 語彙
determiner --> [the].   adjective --> [decorated].
determiner --> [a].
noun --> [pieplate].    verb --> [surprise].
noun --> [surprise].

% Asserting clauses for user:sentence/2


% Asserting clauses for user:noun_phrase/2


% Asserting clauses for user:noun_phrase2/2


% Asserting clauses for user:verb_phrase/2


% Asserting clauses for user:determiner/2


% Asserting clauses for user:adjective/2


% Asserting clauses for user:noun/2


% Asserting clauses for user:verb/2


分脈自由文法は、S\S0（ヘッドとテール）の差分リストを引数とする。prolog文に変換されます。
```prolog
sentence --> noun_phrase, verb_phrase.
```
これをそのままprolog規則に変換すると
```prolog
sentence(S) :- append(NP, VP, S), noun_pharase(NP), verb_phrase(VP).
```
となるが、appendの処理効率がわるいため、差分リストで表すと以下の様になります。
```prolog
sentence(S\S0) :- noun_phrase(S\S1), verb_phrase(S1\S0).
```
さらにAs\Bsは、headとtailのリストに分解されるので、
```prolog
sentence(S, S0) :- noun_phrase(S, S1), verb_phrase(S1, S0).
```
となります。

listing関数を使って、sentenceの変換結果を見ると、上記の変換が行われていることが確認できます。

In [2]:
?- listing(sentence).

:- dynamic sentence/2.

sentence(A, C) :-
    noun_phrase(A, B),
    verb_phrase(B, C).


true

### 文法規則からProlog節への変換
swi-prologは、すでに文法規則からProlog節への変換が組み込まれていますが、Program 16.2の例題の動作をswi-prologで動かしてみます。

sequenceはswi-prologのライブラリをそのまま使用します。

In [3]:
% translate(GrammerRule, PrologClause) :-
%   PrologClauseは、文脈自由文法の規則GrammerRuleと等価なPrologプログラムである。
translate((Lhs --> Rhs), (Head :- Body)) :- 
    translate(Lhs, Head, [Xs, Ys]), translate(Rhs, Body, [Xs, Ys]).
translate((A, B), (A1, B1), [Xs, Ys]) :- 
    translate(A, A1, [Xs, Xs1]), translate(B, B1, [Xs1, Ys]).
translate(A, A1, S) :-
    non_terminal(A), functor(A1, A, 1), arg(1, A1, S).
translate(Xs, true, S) :-
    terminals(Xs), sequence(Xs, S).
non_terminal(A) :- atom(A).
terminals([X|Xs]).
%sequence([X|Xs], [[X|S], S0]) :- sequence(Xs, [S, S0]).
%sequence([], [Xs, Xs]).

% Asserting clauses for user:translate/2


% Asserting clauses for user:translate/3


% Asserting clauses for user:terminals/1


動作確認として、以下の自由文脈文法をProlog節に変換してみます。
```
s --> n, v
```

In [4]:
?- translate((s --> n, v), (H :- B)), print(H), print(:-), print(B).

s([_6856,_6862]):-n([_6856,_6896]),v([_6896,_6862])

H = s([_6856,_6862]),
B = n([_6856,_6896]),v([_6896,_6862])

出力されたProlog節を実行し、listing関数で表示してみると、期待通り以下の出力が返されます。
```
s([A, C]) :- n([A, B]), v([B, C]).

In [5]:
s([_9348,_9354]):-n([_9348,_9388]),v([_9388,_9354])

% Asserting clauses for user:s/1


In [6]:
?- listing(s).

:- dynamic s/1.

s([A, C]) :-
    n([A, B]),
    v([B, C]).


true

### 差分リストの表現
15章のProgram 15.2では差分リストを使って、以下のようにappend_dlが定義されています。

```prolog
append_dl(As, Bs, Cs) :- append_dl(Xs\Ys, Ys\Zs, Xs\Zs).
```

このX\Yの部分を[X, Y]に置き換えて、動きを見てみます。

In [7]:
append_dl([Xs, Ys], [Ys, Zs], [Xs, Zs]).
?- append_dl([[a, b, c|Xs], Xs], [[1, 2], []], Ys).

% Asserting clauses for user:append_dl/3


Xs = [1,2],
Ys = [[a,b,c,1,2],[]]

テキストの本文に?- append_dl([a, b, c|Xs]\Xs, [1, 2]\[], Ys)の結果が(Xs=[1,2], Ys=[a, b, c, 1, 2]\[])と記述があります。

これは上記の結果と合致しています。

### 16.5の例題
数字を英語読みに変換する16.5の例題をswi-prologで試してみます。

In [8]:
number(0) --> [zero].
number(N) --> xxx(N).
xxx(N) --> digit(D), [hundred], rest_xxx(N1), { N is D*100 + N1}.
xxx(N) --> xx(N).
rest_xxx(0) --> [].
rest_xxx(N) --> [and], xx(N).

xx(N) --> digit(N).
xx(N) --> teen(N).
xx(N) --> tens(T), rest_xx(N1), {N is T + N1}.

rest_xx(0) --> [].
rest_xx(N) --> digit(N).

digit(1) --> [one].     teen(10) --> [ten].
digit(2) --> [two].     teen(11) --> [eleven].
digit(3) --> [three].   teen(12) --> [twelve].
digit(4) --> [four].    teen(13) --> [thirteen].
digit(5) --> [five].    teen(14) --> [fourteen].
digit(6) --> [six].     teen(15) --> [fifteen].
digit(7) --> [seven].   teen(16) --> [sixteen].
digit(8) --> [eight].   teen(17) --> [seventeen].
digit(9) --> [nine].    teen(18) --> [eighteen].
                        teen(19) --> [nineteen].
tens(20) --> [twenty].
tens(30) --> [thirty].
tens(40) --> [fourty].
tens(50) --> [fifty].
tens(60) --> [sixty].
tens(70) --> [seventy].
tens(80) --> [eighty].
tens(90) --> [ninety].


% Asserting clauses for user:number/3


% Asserting clauses for user:xxx/3


% Asserting clauses for user:rest_xxx/3


% Asserting clauses for user:xx/3


% Asserting clauses for user:rest_xx/3


% Asserting clauses for user:digit/3


% Asserting clauses for user:teen/3


% Asserting clauses for user:tens/3


試しに66をnumberに渡してみます。期待通りsixty sixがリストして返されました。

In [9]:
?- number(66, Ns, []).

Ns = [sixty,six]

## 和と積の演算をDCGで表す
和と積の演算をDefinite Clause Grammer(DCG)で実装してみましょう。
DCGへの入力は、終端子、非終端子の並びをカンマで区切ったリストで与えます。

以下が、和と積の演算（優先順位を考慮）をDCGで定義したものです。
ここでの表現で空（空のリスト）または、演算子とexprの連結として表現している点です。

In [10]:
expr --> term, addterm.
addterm --> [].
addterm --> [+], expr.
term --> factor, multfactor.
multfactor --> [].
multfactor --> [*], term.
factor --> [I], {integer(I)}.
factor --> ['('], expr, [')'].

% Asserting clauses for user:expr/2


% Asserting clauses for user:addterm/2


% Asserting clauses for user:term/2


% Asserting clauses for user:multfactor/2


% Asserting clauses for user:factor/2


試しに4*5+1がexprとして成り立っているかexprに問い合わせてみましょう。

In [11]:
?- expr([4,*,5,+,1], []).

true

### 計算結果を返す
電卓のように計算結果を返すようにDCGを修正してみましょう。
途中の{}で囲まれた箇所がprologの実行文として処理されます。

Xが項(term)でYがaddtermなら、NにX + Yの結果をセットすると言うのが、最初の文の読み方となります。

```prolog
expr(N) --> term(X), addterm(Y), { N is X + Y}.
```

prologの処理を追加したDCGは、以下のようになります。

In [12]:
expr(N) --> term(X), addterm(Y), { N is X + Y}.
addterm(0) --> [].
addterm(N) --> [+], expr(N).
term(N) --> factor(X), multfactor(Y), {N is X * Y}.
multfactor(1) --> [].
multfactor(S) --> [*], term(S).
factor(I) --> [I], {integer(I)}.

% Asserting clauses for user:expr/3


% Asserting clauses for user:addterm/3


% Asserting clauses for user:term/3


% Asserting clauses for user:multfactor/3


% Asserting clauses for user:factor/3


この文法のprologに変換した結果をlisting関数を使って確認してみましょう。

exprの2つ目の定義には、期待通りC is D+Eが計算されています。

In [13]:
?- listing(expr).

:- dynamic expr/2.

expr(A, C) :-
    term(A, B),
    addterm(B, C).

:- dynamic expr/3.

expr(C, A, F) :-
    term(D, A, B),
    addterm(E, B, G),
    C is D+E,
    F=G.


true

それではexprを使って計算結果を見てみましょう。
積が和よりも優先して処理され、演算結果が７になっていることが確認できました。

In [14]:
?- expr(Y, [1,+,2,*,3], []).

Y = 7

### ANTRLの四則演算をprologで実装
私のブログにANTRLを使って四則演算をする例題があります。
- <a href="https://take-pwave.sakura.ne.jp/index.php?antlr%2FANTLRWorks%E3%82%92%E4%BD%BF%E3%81%A3%E3%81%A6%E3%81%BF%E3%82%8B">antlr/ANTLRWorksを使ってみる</a>

ここで紹介している文法をprologにDCGに書き換えてみます。

```
expression
	: product ('+' product | '-' product)*
	;

product
	: power ('*' power | '/' power)*
	;
power
	: factor ('^' power)?
	;
factor
	: IDENTIFIER
	| CONSTANT
	| '(' expression ')'
	;
```

In [15]:
expression(N) -->
    product(X), 
    ( [],
        {N is X}
    | [+], expression(Y), 
        {N is X + Y} 
    | [-], expression(Y), 
        {N is X - Y}
    ).
product(N) -->
    power(X), 
    ( [],
        { N is X}
    | [*], product(Y),
        { N is X * Y}
    | [/], product(Y),
        { N is X / Y}
    ).
power(N) -->
    factor(X),
    ([],
        { N is X}
    | ['^'], power(Y),
        { N is X**Y}
    ).
factor(I) --> [I], {number(I)}.
    

% Asserting clauses for user:expression/3


% Asserting clauses for user:product/3


% Asserting clauses for user:power/3


Previously defined clauses of user:factor/3 were retracted:
:- dynamic factor/3.

factor(A, [A|C], B) :-
    integer(A),
    B=C.

% Asserting clauses for user:factor/3


これで1+2^3*4を計算してみましょう。

優先順位的には、1 + ((2^3)*4)となりますので、$1 + (8*4) = 33$となります。

In [16]:
?- expression(N, [1, +, 2, '^', 3, *, 4], []).

N = 33

19章のメタ・インタプリタを使って、途中経過を出力すると以下のようになります。

```text
node(+,                                 => +                                    => 1 + 32   => 33
    node(product,
        node(power,
            node(factor,1)              => 1                                    ↑
        )
    ),
    node(product,
        node(*,                         => *                    => 8*4  => 32   ↑
            node(^,                     => ^    => 2^3  => 8    ↑
                node(factor,2),         => 2    ↑
                    node(power,         
                        node(factor,3)  => 3    ↑
                    )
                ),
                node(product,
                    node(power,
                        node(factor,4)  => 4                    ↑
                    )
                )
            )
        )
    )
```

## 構文木の生成
構文解析の結果として、構文木の生成があります。以前、ANTLRというツールを使って四則演算の構文木生成を試した記事が以下にあります。
−　<a href="https://take-pwave.sakura.ne.jp/index.php?antlr%2F%E6%A7%8B%E6%96%87%E6%9C%A8%E3%82%92%E4%BD%BF%E3%81%A3%E3%81%9F%E8%A7%A3%E6%9E%90">antlr/構文木を使った解析</a>

コンパイラーやインタプリターは、生成された構文木を使って生成できます。

### Prologで構文木生成を試す
四則演算構文から入力式に対する構文木生成をPrologを使って試してみます。

以下に四則演算をexpressionとして定義し、その構文解析をparseと定義します。

In [17]:
parse(Source, Structure) :- expression(Structure, Source, []).
expression(N) -->
    product(X),         
    ( [],
        { N = node("product", X) }
    | [+], expression(Y1), 
        { N = node("+", X, Y1) }
    | [-], expression(Y2), 
        { N = node("-", X, Y2) }
    ).
product(N) -->
    power(X),
    ( [],
        { N = node("product", X) }
    | [*], product(Y1),
        { N = node("*", X, Y1) }
    | [/], product(Y2),
        { N = node("/", X, Y2) }
    ).
power(N) -->
    factor(X),
    ([],
        { N = node("power", X) }
    | ['^'], power(Y1),
        { N = node("^", X, Y1)}
    ).
factor(node("factor", I)) --> [I], {number(I)}.

% Asserting clauses for user:parse/2


Previously defined clauses of user:expression/3 were retracted:
:- dynamic expression/3.

expression(C, A, E) :-
    product(D, A, B),
    (   B=F,
        C is D,
        E=F
    ;   B=[+|G],
        expression(H, G, I),
        C is D+H,
        E=I
    ;   B=[-|J],
        expression(H, J, K),
        C is D-H,
        E=K
    ).

% Asserting clauses for user:expression/3


Previously defined clauses of user:product/3 were retracted:
:- dynamic product/3.

product(C, A, E) :-
    power(D, A, B),
    (   B=F,
        C is D,
        E=F
    ;   B=[*|G],
        product(H, G, I),
        C is D*H,
        E=I
    ;   B=[/|J],
        product(H, J, K),
        C is D/H,
        E=K
    ).

% Asserting clauses for user:product/3


Previously defined clauses of user:power/3 were retracted:
:- dynamic power/3.

power(C, A, E) :-
    factor(D, A, B),
    (   B=F,
        C is D,
        E=F
    ;   B=[^|G],
        power(H, G, I),
        C is D**H,
        E=I
    ).

% Asserting clauses for user:power/3


Previously defined clauses of user:factor/3 were retracted:
:- dynamic factor/3.

factor(A, [A|C], B) :-
    number(A),
    B=C.

% Asserting clauses for user:factor/3


In [18]:
?- parse([1, +, 2, '^', 3, *, 4], Tree).

Tree = node(+,node(product,node(power,node(factor,1))),node(product,node(*,node(^,node(factor,2),node(power,node(factor,3))),node(product,node(power,node(factor,4))))))

このままですと読み辛いので、出力を手で加工します。

これに$1+2^3*4$のトークンを入力入力し、生成された構文を見てみましょう。
nodeは、オペレータと左項、右項をもつ

```
Tree = node(+,
node(product,
node(power,
node(factor,1)))    ,node(product,
                    node(*,         node(^,
                                    node(factor,2), node(power,
                                                    node(factor,3))),   node(product,
                                                                        node(power,
                                                                        node(factor,4))))))

```
となり、終端記号だけを残すと
```
Tree = node(+,
        1,      node(*,     
                node(^, 2, 3),   4))
```
となり、優先順位が考慮された構文木となっていることが分かります。

## 23章　コンパイラをswi prologで試す
ここからは、23章のコンパイラの作成をswi prologを使って試してみます。



最初にコンパイラを以下の要素で定義します。
- 構文解析（parse）
- エンコーダー（encode）
- アセンブリ（assemble）

In [20]:
compile(Tokens, ObjectCode) :-
    parse(Tokens, Structure),
    encode(Structure, Dictionary, Code),
    assemble(Code, Dictionary, ObjectCode).

% Asserting clauses for user:compile/2


### 構文解析の作成
構文解析（parse）は、入力されたプログラムを解析し、構文木を生成します。

In [21]:
parse(Source, Structure) :-
    pl_program(Structure, Source, []).
pl_program(S) --> [program], identifier(X), [';'], statement(S).

Previously defined clauses of user:parse/2 were retracted:
:- dynamic parse/2.

parse(B, A) :-
    expression(A, B, []).

% Asserting clauses for user:parse/2


% Asserting clauses for user:pl_program/3


今回作成する言語の文法は、以下のように定義されています。

四則演算や論理演算子の優先順位は考慮されていないきわめて単純な文法で構成されています。

In [22]:
statement((S ; Ss)) -->
    [begin], statement(S), rest_statements(Ss).
statement(assign(X, V)) -->
    identifier(X), [':='], expression(V).
statement(if(T, S1, S2)) -->
    [if], test(T), [then], statement(S1), [else], statement(S2).
statement(while(T, S)) -->
    [while], test(T), [do], statement(S).
statement(read(X)) -->
    [read], identifier(X).
statement(write(X)) -->
    [write], expression(X).

rest_statements((S ; Ss)) --> 
    [';'], statement(S), rest_statements(Ss).
rest_statements(void) --> [end].

expression(X) --> pl_constant(X).
expression(expr(Op, X, Y)) -->
    pl_constant(X), arithmetic_op(Op), expression(Y).

arithmetic_op('+') --> ['+'].
arithmetic_op('-') --> ['-'].
arithmetic_op('*') --> ['*'].
arithmetic_op('/') --> ['/'].

pl_constant(name(X)) --> identifier(X).
pl_constant(number(X)) --> pl_integer(X).

identifier(X) --> [X], { atom(X) }.
pl_integer(X) --> [X], { integer(X) }.

test(compare(Op, X, Y)) -->
    expression(X), comparision_op(Op), expression(Y).

comparision_op('=') --> ['='].
comparision_op('<>') --> ['<>'].
comparision_op('>') --> ['>'].
comparision_op('<') --> ['<'].
comparision_op('>=') --> ['>='].
comparision_op('<=') --> ['<='].

% Asserting clauses for user:statement/3


% Asserting clauses for user:rest_statements/3


Previously defined clauses of user:expression/3 were retracted:
:- dynamic expression/3.

expression(C, A, E) :-
    product(D, A, B),
    (   B=F,
        C=node("product", D),
        E=F
    ;   B=[+|G],
        expression(H, G, I),
        C=node("+", D, H),
        E=I
    ;   B=[-|J],
        expression(K, J, L),
        C=node("-", D, K),
        E=L
    ).

% Asserting clauses for user:expression/3


% Asserting clauses for user:arithmetic_op/3


% Asserting clauses for user:pl_constant/3


% Asserting clauses for user:identifier/3


% Asserting clauses for user:pl_integer/3


% Asserting clauses for user:test/3


% Asserting clauses for user:comparision_op/3


次に、構文木からアセンブリ言語を生成するエンコーダを作成します。

In [23]:
encode((X;Xs), D, (Y;Ys)) :-
    encode(X, D, Y), encode(Xs, D, Ys).
encode(void, D, no_op).
encode(assign(Name, E), D, (Code; instr(store, Address))) :-
    lookup(Name, D, Address), encode_expression(E, D, Code).
encode(if(Test, Then, Else), D, (TestCode; ThenCode; instr(jump, L2); label(L1); ElseCode; label(L2))) :-
    encode_test(Test, L1, D, TestCode),
    encode(Then, D, ThenCode),
    encode(Else, D, ElseCode).
encode(while(Test, Do), D, (label(L1); TestCode; DoCode; instr(jump, L1); label(L2))) :-
    encode_test(Test, L2, D, TestCode),
    encode(Do, D, DoCode).
encode(read(X), D, instr(read, Address)) :-
    lookup(X, D, Address).
encode(write(E), D, (Code; instr(write, 0))) :-
    encode_expression(E, D, Code).

% Asserting clauses for user:encode/3


In [24]:
encode_expression(number(C), D, instr(loadc, C)).
encode_expression(name(X), D, instr(load, Address)) :-
    lookup(X, D, Address).
encode_expression(expr(Op, E1, E2), D, (Load; Instruction)) :-
    single_instruction(Op, E2, D, Instruction),
    encode_expression(E1, D, Load).
encode_expression(expr(Op, E1, E2), D, Code) :-
    not(single_instruction(Op, E2, D, Instruction)),
    single_operation(Op, E1, D, E2Code, Code),
    encode_expression(E2, D, E2Code).
single_instruction(Op, number(C), D, instr(OpCode, C)) :-
    literal_operation(Op, OpCode), lookup(X, D, A).
single_instruction(Op, name(X), D, instr(OpCode, A)) :-
    memory_operation(Op, OpCode),
    lookup(X, D, A).
single_operation(Op, E, D, Code, (Code; Instruction)) :-
    commutative(Op),
    single_instruction(Op, E, D, Instruction).
single_operation(Op, E, D, Code, (Code; instr(store, Address); Load; instr(OpCode, Address))) :-
    not(commutative(Op)),
    lookup('temp', D, Address),
    encode_expression(E, D, Load),
    op_code(OP, E, OpCode).
op_code(Op, number(C), OpCode) :- 
    literal_operation(Oop, OpCode).
op_code(Op, name(X), OpCode) :-
    memory_operation(Op, OpCode).

% Asserting clauses for user:encode_expression/3


% Asserting clauses for user:single_instruction/4


% Asserting clauses for user:single_operation/5


% Asserting clauses for user:op_code/3


In [25]:
literal_operation('+', addc).       memory_operation('+', add).
literal_operation('-', subc).       memory_operation('-', sub).
literal_operation('*', mulc).       memory_operation('*', mul).
literal_operation('.', divc).       memory_operation('/', div).
commutative('+').                   commutative('*').

encode_test(compare(Op, E1, E2), Label, D, (Code; instr(OpCode, Label))) :-
    comparison_opcode(Op, OpCode),
    encode_expression(expr('-', E1, E2), D, Code).
% 逆のオペコードを出力している
comparison_opcode('=', jumpne).     comparison_opcode('<>', jumpeq).
comparison_opcode('>', jumple).     comparison_opcode('>=', jumplt).
comparison_opcode('<', jumpge).     comparison_opcode('<=', jumpgt).

%swi-prologではインスタンス化されていない変数との比較は、@<を使用する
lookup(Key, dict(Key, X, Left, Right), Value) :-
    !, X = Value.
lookup(Key, dict(Key1, X, Left, Right), Value) :-
    Key @< Key1, lookup(Key, Left, Value).
lookup(Key, dict(Key1, X, Left, Right), Value) :-
    Key1 @< Key, lookup(Key, Right, Value).


% Asserting clauses for user:literal_operation/2


% Asserting clauses for user:memory_operation/2


% Asserting clauses for user:commutative/1


% Asserting clauses for user:encode_test/4


% Asserting clauses for user:comparison_opcode/2


% Asserting clauses for user:lookup/3


### アセンブリの作成
最後にアセンブラからマシン語を生成するアセンブリを作成します。

In [26]:
assemble(Code, Dictionary, TidyCode) :-
    tidy_and_count(Code, 1, N, TidyCode, (instr(halt, 0); block(L))),
    N1 is N + 1,
    allocate(Dictionary, N1, N2),
    L is N2 - N1, !.
tidy_and_count((Code1; Code2), M, N, TCode1, TCode2) :-
    tidy_and_count(Code1, M, M1, TCode1, Rest),
    tidy_and_count(Code2, M1, N, Rest, TCode2).
tidy_and_count(instr(X, Y), N, N1, (instr(X, Y); Code), Code) :-
    N1 is N + 1.
tidy_and_count(label(N), N, N, Code, Code).
tidy_and_count(no_op, N, N, Code, Code).
allocate(void, N, N).
allocate(dict(Name, N1, Before, After), N0, N) :-
    allocate(Before, N0, N1),
    N2 is N1 + 1,
    allocate(After, N2, N).

% Asserting clauses for user:assemble/3


% Asserting clauses for user:tidy_and_count/5


% Asserting clauses for user:allocate/3


## 動作確認
コンパイラーの動作確認していきます。

最初に構文解析（parse）の動きを確認します。最初はもっとも単純な文法で動きを確認します。

```Pascal
program test0;
begin
    x := 1
end
```

In [27]:
?- parse([program, test0, ';', begin, x, ':=', 1, end], S).

S = assign(x,number(1));void

### テキストの例題の動作確認
次に、23章で紹介されている例題をコンパイルしてみます。

最初にtest1をコンパイルします。

In [28]:
?- parse([program, test1, ';', begin, write, x, '+', y, '-', z, '/', 2, end], S), encode(S, Dict, RCode), assemble(RCode, Dict, OpCode).

S = write(expr(+,name(x),expr(-,name(y),expr(/,name(z),number(2)))));void,
Dict = dict(x,12,dict(temp,11,void,void),dict(y,13,void,dict(z,14,void,void))),
RCode = ((((instr(loadc,2);instr(store,11);instr(load,14);instr(add,11));instr(store,11);instr(load,13);instr(add,11));instr(add,12));instr(write,0));no_op,
OpCode = instr(loadc,2);instr(store,11);instr(load,14);instr(add,11);instr(store,11);instr(load,13);instr(add,11);instr(add,12);instr(write,0);instr(halt,0);block(4)

次に、test2をコンパイルします。

In [29]:
?- parse([program, test2, ';', begin, if, a, '>', b, then, max, ':=', a, else, max, ':=', b, end], S), encode(S, Dict, RCode), assemble(RCode, Dict, OpCode).

S = if(compare(>,name(a),name(b)),assign(max,name(a)),assign(max,name(b)));void,
Dict = dict(b,11,dict(a,10,void,void),dict(max,12,void,void)),
RCode = (((instr(load,10);instr(sub,11));instr(jumple,7));(instr(load,10);instr(store,12));instr(jump,9);label(7);(instr(load,11);instr(store,12));label(9));no_op,
OpCode = instr(load,10);instr(sub,11);instr(jumple,7);instr(load,10);instr(store,12);instr(jump,9);instr(load,11);instr(store,12);instr(halt,0);block(3)

最後に、test3をコンパイルします。

In [30]:
?- parse(
    [program, factorial, ';',
        begin,
            read, value, ';',
            count, ':=', 1, ';',
            result, ':=', 1, ';',
            while, count, '<', value, do,
                begin,
                    count, ':=', count, '+', 1, ';',
                    result, ':=', result, '*', count,
                end, ';',
            write, result,
        end], S), encode(S, Dict, RCode), assemble(RCode, Dict, OpCode).

S = read(value);assign(count,number(1));assign(result,number(1));while(compare(<,name(count),name(value)),(assign(count,expr(+,name(count),number(1)));assign(result,expr(*,name(result),name(count)));void));write(name(result));void,
Dict = dict(value,21,dict(count,19,void,dict(result,20,void,void)),void),
RCode = instr(read,21);(instr(loadc,1);instr(store,19));(instr(loadc,1);instr(store,20));(label(6);((instr(load,19);instr(sub,21));instr(jumpge,16));(((instr(load,19);instr(addc,1));instr(store,19));((instr(load,20);instr(mul,19));instr(store,20));no_op);instr(jump,6);label(16));(instr(load,20);instr(write,0));no_op,
OpCode = instr(read,21);instr(loadc,1);instr(store,19);instr(loadc,1);instr(store,20);instr(load,19);instr(sub,21);instr(jumpge,16);instr(load,19);instr(addc,1);instr(store,19);instr(load,20);instr(mul,19);instr(store,20);instr(jump,6);instr(load,20);instr(write,0);instr(halt,0);block(3)

構文解析の結果を字下げをつけて読みやすくすると、
```prolog
read(value);assign(count,number(1));assign(result,number(1));
    while(compare(<,name(count),name(value)),
        (assign(count,expr(+,name(count),number(1)));
        assign(result,expr(*,name(result),name(count)));void));
        write(name(result));void
```

出力されたコードを読みやすくすると、以下の様になります。この出力が図23.3と対応しています。
```text
instr(read,21);
instr(loadc,1);
instr(store,19);
instr(loadc,1);
instr(store,20);
instr(load,19);
instr(sub,21);
instr(jumpge,16);
instr(load,19);
instr(addc,1);
instr(store,19);
instr(load,20);
instr(mul,19);
instr(store,20);
instr(jump,6);
instr(load,20);
instr(write,0);
instr(halt,0);
block(3)
```

### 字句解析
これまで、構文解析にはトークンの配列を渡してきました。
実際のコンパイラでは、解析対象のプログラムコードをそのまま渡すために、トークンを切り分ける字句解析が必要になります。

swi prologのtokenizeライブラリを使ってプログラムを字句解析します。

注意する点は、2文字以上のシンボルを1個のトークンに付け替えることです。

In [42]:
% ２文字以上のシンボルトークンを付け替える
tokens([:= | Z]) --> [:], [=], tokens(Z).
tokens([<> | Z]) --> [<], [>], tokens(Z).
tokens([<= | Z]) --> [<], [=], tokens(Z).
tokens([>= | Z]) --> [>], [=], tokens(Z).
tokens([X | Z]) --> [C], tokens(Z), {X = C}.
tokens([]) --> [].

Previously defined clauses of user:tokens/3 were retracted:
:- dynamic tokens/3.

tokens([<>|B], [:|A], D) :-
    A=[=|C],
    tokens(B, C, D).
tokens([<>|B], [<|A], D) :-
    A=[>|C],
    tokens(B, C, D).
tokens([<=|B], [<|A], D) :-
    A=[=|C],
    tokens(B, C, D).
tokens([>=|B], [>|A], D) :-
    A=[=|C],
    tokens(B, C, D).
tokens([C|A], [D|B], E) :-
    tokens(A, B, F),
    C=D,
    E=F.
tokens([], A, A).

% Asserting clauses for user:tokens/3


字句解析lexを以下のように定義します。

In [43]:
lex(File, Tokens) :- open(File, read, In), read_string(In, _, Str), close(In), tokenize_atom(Str, Atoms), tokens(Tokens, Atoms, []).

Previously defined clauses of user:lex/2 were retracted:
:- dynamic lex/2.

lex(A, D) :-
    open(A, read, B),
    read_string(B, _, C),
    close(B),
    tokenize_atom(C, E),
    tokens(D, E, []).

% Asserting clauses for user:lex/2


In [44]:
?- lex('program.txt', Token)

Token = [program,factorial,;,begin,read,value,;,count,:=,1,;,result,:=,1,;,while,count,<,value,do,begin,count,:=,count,+,1,;,result,:=,result,*,count,end,;,write,result,end]

In [45]:
compile(File, ObjectCode) :-
    lex(File, Tokens),
    parse(Tokens, Structure),
    encode(Structure, Dictionary, Code),
    assemble(Code, Dictionary, ObjectCode).

Previously defined clauses of user:compile/2 were retracted:
:- dynamic compile/2.

compile(A, F) :-
    lex(A, B),
    parse(B, C),
    encode(C, E, D),
    assemble(D, E, F).

% Asserting clauses for user:compile/2


In [46]:
?- compile('program.txt', ObjectCode)

ObjectCode = instr(read,21);instr(loadc,1);instr(store,19);instr(loadc,1);instr(store,20);instr(load,19);instr(sub,21);instr(jumpge,16);instr(load,19);instr(addc,1);instr(store,19);instr(load,20);instr(mul,19);instr(store,20);instr(jump,6);instr(load,20);instr(write,0);instr(halt,0);block(3)

出力されたオブジェクトコードをTest３と同じになっています。

```
ObjectCode = 
instr(read,21);
instr(loadc,1);
instr(store,19);
instr(loadc,1);
instr(store,20);
instr(load,19);i
nstr(sub,21);
instr(jumpge,16);
instr(load,19);
instr(addc,1);
instr(store,19);
instr(load,20);
instr(mul,19);
instr(store,20);
instr(jump,6);
instr(load,20);
instr(write,0);
instr(halt,0);
block(3)
```